<!--NAVIGATION-->
< [Préparation](Preparation.ipynb) | [Contenus](Contents.ipynb) | [Condition 2](Condition_2.ipynb) >

# 3. Condition 1: Puits dans les 150 mètres des maisons ou routes
## 3.1 Introduction

Maintenant que nous avons converti les fichiers GeoTIFF au format PCRaster et que nous avons vérifié leurs propriétés, il est temps de procéder à l’analyse des trois conditions:

* Condition 1: Puits à moins de 150 mètres des maisons ou des routes 
* Condition 2: Pas d’industrie, de mine ou de décharge à moins de 300 mètres des puits 
* Condition 3: Puits de moins de 40 mètres de profondeur

Pour la condition 1, nous examinerons d’abord les maisons et nous procéderons comme suit:

1. Créez un calque booléen avec ‘True’ pour les maisons et ‘False’ pour les autres bâtiments 
2. Créer des zones de 150 mètres autour des maisons

Ensuite, nous répéterons les étapes pour les routes.

## 3.2 Créer un calque booléen avec True pour les maisons et False pour les autres bâtiments
Nous tapons généralement les commandes dans l’invite Python. Dans ce Jupyter Notebook, nous simulons cela. C’est-à-dire que tout ce que vous tapez dans les champs est égal à une commande dans l’invite Python.

Chargeons d’abord le module `pcraster`.

In [ ]:
from pcraster import *

N’oubliez pas que nos données sont stockées dans le dossier `data`, tandis que notre Python s’exécute dans le dossier parent. Pour plus de commodité, il est utile de passer au dossier Data afin que tous nos fichiers d’entrée et de sortie y soient lus et écrits respectivement.

Avec la bibliothèque `os`, nous pouvons utiliser la ligne de commande commande.

In [ ]:
import os

Vérifions le répertoire de travail actuel.

In [ ]:
print(os.getcwd())

Nous pouvons utiliser un équivalent de la commande `cd`  pour changer le répertoire: `os.chdir(stringWithPath)`, où `stringWithPath` définit le chemin.

In [ ]:
os.chdir("./data")

Vérifiez où nous en sommes maintenant. Assurez-vous que nous sommes dans le dossier `data`  de ce tutoriel.

In [ ]:
print(os.getcwd())

Nous sommes maintenant prêts à lire `buildg.map` à partir du disque et à l’utiliser pour l’algèbre cartographique avec PCRaster. 
Nous pouvons lire des cartes en utilisant `readmap(stringWithPathToFile)`.

In [ ]:
Buildings = readmap("buildg.map")

Maintenant, nous avons lu `buildg.map` à partir du disque et pouvons-nous y référer avec la variable `Buildings`. 
Il existe différentes façons de visualiser la carte:  
* En Python, nous pouvons utiliser l’opérateur `aguila`: `aguila(Buildings)` 
* À l’invite de commande, nous pouvons utiliser la commande 'Aguila' avec le nom de fichier: `aguila buildg.map`  
* Utilisez des outils de traçage Python qui fonctionnent également avec les notebooks Jupyter sur Binder, etc.  

Nous allons maintenant utiliser la dernière option de ce notebook Jupyter. Mais vous pouvez également essayer les autres si vous exécutez ce notebook localement.

Nous pouvons utiliser l’opérateur `plot` pour visualiser les cartes PCRaster. Il a la syntaxe suivante:

```Python
plot(raster, labels=None, title=None, filename=None)
```

Les entrées sont:

`raster`: Carte raster avec le type Boolean, Nominal, Scalar ou Ldd.

`labels`: Optionnel. Dictionnaire des étiquettes qui doivent être utilisées pour la légende, les valeurs de cellule seront utilisées autrement.

`title`: Optionnel. Titre de la légende sous forme de chaîne, essaie d’identifier le nom de la variable autrement.

`filename`: Optionnel. Si le tracé fourni sera écrit sur le disque.

Les données raster n’ont pas de table attributaire. Les valeurs du raster `Buildings`  représentent les classes suivantes :

| Value | Class |
|:-------:|:-------:|
| 0 | None |
| 1 | House |
| 2 | Public building |
| 3 | Landfill |
| 4 | Industry |
| 5 | Mine |

Nous pouvons enregistrer la légende dans un dictionnaire Python:

In [ ]:
Legend = {0:"None",1:"House",2:"Public building",3:"Landfill",4:"Industry",5:"Mine"}

Nous pouvons maintenant mettre en graphique la couche Bâtiments en exécutant ce code :

In [ ]:
plot(Buildings, labels=Legend, title="Buildings", filename=None)

Nous devons d’abord créer une couche booléenne avec ‘True’ pour les maisons et ‘False’ pour les autres bâtiments.

In [ ]:
Houses = Buildings == 1

Ici nous avons utilisé l'opérateur `==`. Cela crée une carte booléenne avec la valeur 1 pour toutes les cellules qui sont égales à 1 et 0 pour toutes les autres cellules. Vous trouverez ici plus d’informations sur [operateurs de PCRaster](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/python/quickstart.html#operators).
Visualisons le résultat avec l’opérateur `plot`:

## 3.3 Créer des zones de 150 mètres autour des maisons
L’étape suivante consiste à créer des zones de 150 m autour des maisons.

Vous pouvez utiliser la [fonction `spreadmaxzone`](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_spreadmaxzone.html).

La syntaxe est `Result = spreadmaxzone(points, initialfrictiondist, friction, max_distance)`
Dans notre cas `points` sont les maisons `Houses`, `initialfrictiondist = 0`, `friction = 1` and `max_distance = 150`. `intialfrictiondistance = 0` signifie qu’il n’y a pas de friction à l’emplacement des maisons et avec `friction = 1` nous calculons la distance à partir des autres pixels de manière égale sans poids différents.

In [ ]:
houses150m = spreadmaxzone(Houses, 0, 1, 150)

Visualisez le résultat.

Nous pouvons enregistrer ce résultat sur le disque à l’aide de la fonction `rapport`. Le premier argument est le nom de la variable de la carte que vous souhaitez écrire sur le disque. Le deuxième argument est la chaîne du nom du fichier. 

In [ ]:
report(houses150m,"houses150m.map")

Si vous exécutez ce notebook localement, vérifiez que `houses150m.map` est stocké dans le dossier `data` et visualisez le résultat à l’aide d’aguila à partir de l’invite de commande :
```
aguila houses150.map
```

## 3.4 Créer des zones de 150 mètres autour des routes
De la même manière, nous pouvons maintenant calculer la zone tampon de 150 m autour des routes.

Les valeurs dans `roads.map` représentent les classes suivantes: 

| Value | Class |
|:-------:|:-------:|
| 0 | No road |
| 1 | Dirt road |
| 2 | Tarmac |

Faites les mêmes étapes que pour `houses150m` et enregistrez le résultat dans `roads150m.map`.

Visualisez le résultat.

Enregistrez le résultat sur le disque avec le nom `roads150m.map`.

<!--NAVIGATION-->
< [Préparation](Preparation.ipynb) | [Contenus](Contents.ipynb) | [Condition 2](Condition_2.ipynb) >